## Dependencias

In [1]:
%pip install mediapipe opencv-python pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import csv
import os

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

## Generación de Dataset .CSV

### Versión Inicial

In [ ]:
class_name = "A"
#class_name = "B"
#class_name = "C"
#class_name = "D/1"
#class_name = "E"
#class_name = "F"
#class_name = "G"
#class_name = "H"
#class_name = "I"
#class_name = "K"
#class_name = "L"
#class_name = "M"
#class_name = "N"
#class_name = "O/0"
#class_name = "P"
#class_name = "Q"
#class_name = "R"
#class_name = "S"
#class_name = "T"
#class_name = "U"
#class_name = "V/2"
#class_name = "W"
#class_name = "X"
#class_name = "Y"

In [ ]:
#class_name = "0"
#class_name = "1"
#class_name = "2"
#class_name = "3"
#class_name = "4"
#class_name = "5"
#class_name = "6"
#class_name = "7"
#class_name = "8"
#class_name = "9"

In [ ]:
# Crear o inicializar el archivo CSV
with open('last_model_v01.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ['class'] + [f'{axis}{i+1}' for i in range(21) for axis in ['x', 'y', 'z']]
    csv_writer.writerow(header)

In [ ]:
header

In [ ]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1) 
        image.flags.writeable = False
        
        results = hands.process(image)
        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                if handedness.classification[0].label == 'Right':
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                    )

        try:
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]).flatten())
            pose_row.insert(0, class_name)
            
            with open('last_model_v01.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(pose_row) 
            
        except:
            pass

        cv2.imshow('Realiza la pose a guardar', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

### Captura de imágenes y organzación de carpetas

In [4]:
output_dir = "dataset_mst"
classes = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
os.makedirs(output_dir, exist_ok=True)

for class_name in classes:
    os.makedirs(os.path.join(output_dir, class_name), exist_ok=True)

In [4]:
output_csv = "letters_mst.csv" # Dataset a crear.

In [52]:
# Configuración inicial (Los dos primero deben ser igual de clase)
dataset_folder = "dataset_mst/Z" # Carpeta donde están las imágenes a analizar.
class_name = "Z" # Clase de la carpeta a analizar.

### Generador img a carpetas

In [9]:
cap = cv2.VideoCapture(1)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
        image = cv2.flip(image, 1)
        image.flags.writeable = False 

        results = hands.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        cv2.imwrite(os.path.join(
                        dataset_folder,
                        '{}.jpg'.format(uuid.uuid1())), 
                    image)
        
        cv2.imshow('Realizar pose de mano a grabar', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

### Procesamiento del dataset y extracción de landmarks

In [53]:
# Verificar si el archivo CSV ya existe y si tiene cabecera
header_written = os.path.exists(output_csv)

# Procesar las imágenes del dataset
with mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.8) as hands:
    for image_name in os.listdir(dataset_folder):
        image_path = os.path.join(dataset_folder, image_name)
        if not os.path.isfile(image_path):
            continue

        # Leer la imagen
        image = cv2.imread(image_path)
        if image is None:
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Obtener los landmarks como lista
                pose_row = list(
                    np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]).flatten()
                )
                pose_row.insert(0, class_name)  # Insertar la clase al inicio de la fila

                # Escribir los landmarks en el archivo CSV
                with open(output_csv, mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

                    if not header_written:
                        header = ['class'] + [f'{axis}{i+1}' for i in range(21) for axis in ['x', 'y', 'z']]
                        csv_writer.writerow(header)
                        header_written = True
                    
                    csv_writer.writerow(pose_row)